In [53]:
# Beautiful Soup Tests

In [54]:
from bs4 import BeautifulSoup
import lxml
from lxml import html
import requests

In [55]:
with open("WebReg_ myCoursebin.html") as fp:
    soup = BeautifulSoup(fp, "lxml")

In [56]:
class course:
    def __init__(self, section, time, availible, scheduled, registered):
        self.section = section
        self.time = time
        self.availible = availible
        self.scheduled = scheduled
        self.registered = registered
        #print("Course " + str(self.section) + " created")
    def unschedule(self, s):
        if self.scheduled:
            domain = "https://webreg.usc.edu"
            req = domain + "/myCoursebin/SchdUnschRmv?act=UnSched&section=" + self.section
            s.get(req)
            self.scheduled = False
            print("Unscheduled with: " + req)
    def schedule(self, s):
        if not self.scheduled:
            domain = "https://webreg.usc.edu"
            req = domain + "/myCoursebin/SchdUnschRmv?act=Sched&section=" + self.section
            s.get(req)
            self.scheduled = True
            print("Scheduled with: " + req)            


In [57]:
# Login

# Returns session
def usc_auth(username, password):

    s = requests.Session()
    r = s.get('https://my.usc.edu/')

    enter_page = lxml.html.fromstring(r.content)
    form_1 = enter_page.xpath("//form[@name='form1']")
    #print form_1[0].attrib['action']

    Query = "https://shibboleth.usc.edu" + form_1[0].attrib['action']
    #print Query
    payload = {'_eventId_proceed' : '', 'shib_idp_ls_exception.shib_idp_persistent_ss' : '',
     'shib_idp_ls_exception.shib_idp_session_ss' : '',
     'shib_idp_ls_success.shib_idp_persistent_ss' : 'false',
     'shib_idp_ls_success.shib_idp_session_ss' : 'false',
     'shib_idp_ls_supported' : '',
     'shib_idp_ls_value.shib_idp_persistent_ss' : '',
     'shib_idp_ls_value.shib_idp_session_ss': ''
     }

    bypass = s.post(Query, data=payload)

    payload = {
        '_eventId_proceed': '',
        'j_password': password,
        'j_username': username
    }

    login = s.post(bypass.url, data=payload)
    tree = lxml.html.fromstring(login.content)

    SAMLResponses = tree.xpath("//form//input[@name='SAMLResponse']")
    SAMLResponse = SAMLResponses[0].attrib['value']

    payload = {'SAMLRequest':SAMLResponse}
    login = s.post('https://my.usc.edu/portal/Shibboleth.sso/SAML2/POST', payload)

    print ("Are We Logged In? : " + str(username in login.text))
    if username not in login.text:
        raise Exception("Login Error")
    return s

In [58]:

# Poll Courses
# get request to "/myCoursebin/SchdUnschRmv?act=UnSched&section=<MYSECTION>"

def course_report(s):
    r = s.get("https://webreg.usc.edu/myCourseBin")
    soup = BeautifulSoup(r.text, "lxml")
    courses = set()
    course_data0 = soup.find_all(class_="section_crsbin_alt0")
    course_data1 = soup.find_all(class_="section_crsbin_alt1")
    course_data2 = soup.find_all(class_="section_crsbin_alt2")
    
    course_data = course_data0 + course_data1 + course_data2
    
    for c in course_data:
        section_full = str(c.contents[3].contents[1].string)
        section = section_full.split(' ',1)[0]

        availible = "Closed" not in c.contents[11].contents[1].string  

        time = c.contents[13].contents[1].string

        status_sY_rN = "block" in c.contents[1].contents[2]['style']
        status_sN_rN = "block" in c.contents[1].contents[4]['style']
        status_sY_rY = "block" in c.contents[1].contents[6]['style']
        status_sN_rY = "block" in c.contents[1].contents[8]['style']

        if status_sY_rN:
            scheduled = True
            registered = False
        elif status_sN_rN:
            scheduled = False
            registered = False
        elif status_sY_rY:
            scheduled = True
            registered = True
        elif status_sN_rY:
            scheduled = False
            registered = True
        else:
            print("An error has ouccured, course bin class " + section + " is corrupt")

        #print ("Section: " + section)
        #print ("Avalible: " + str(availible))
        #print ("Time: " + time)
        #print ("Scheduled: " + str(scheduled))
        #print ("Registered: " + str(registered))
        courses.add(course(section, time, availible, scheduled, registered))
        #print ()
        
    return courses   


In [59]:
import pickle
import os
import re

pickle_file = 'my_usc_session.pkl'
term = 'spring' # Options, Spring Fall Summer

home = 'https://my.usc.edu/'
webreg_connect = 'https://my.usc.edu/portal/oasis/webregbridge.php'
webreg = "https://webreg.usc.edu/"

def new_saved_session(_pickle_file):
    print("No pickle file, generating new session")
    with open(_pickle_file, 'wb') as output:
        s = usc_auth(credentials.get_username(), credentials.get_password())
        pickle.dump(s, output, pickle.HIGHEST_PROTOCOL)
        print("Logged In")


def recover_session(_pickle_file):
    with open(_pickle_file, 'rb') as myinput:
        print("File exists! recovering session ...")
        recovered_s = pickle.load(myinput)
        return recovered_s

# returs session
def webreg_login():
    # Save and recover session
    if not os.path.isfile(pickle_file):
        new_saved_session(pickle_file)
    try:
        s = recover_session(pickle_file)
    except:
        print("Recovery failed.")
        s = new_saved_session(pickle_file)

    terms_page = s.get(webreg_connect)

    session_end_phrase = 'Your session has ended.'
    session_ended = session_end_phrase in terms_page.text

    if session_ended:
        print("Session ended, forcing new session")
        # force new session
        new_saved_session(pickle_file)
        s = recover_session(pickle_file)
        terms_page = s.get(webreg_connect)

    tree = lxml.html.fromstring(terms_page.content)


    if term == 'fall':
        term_element = tree.xpath("//ul//li[@id='termmenuFall']/a")
    elif term == 'summer':
        term_element = tree.xpath("//ul//li[@id='termmenuSumm']/a")
    elif term == 'spring':
        term_element = tree.xpath("//ul//li[@id='termmenuSpr']/a")

    addr = webreg + term_element[0].attrib['href']
    catalogue = s.get(addr)
    return s

In [132]:
def simplify_schedule(s, courses):
    change_log = set()
    for c in courses:
        if c.registered:
            if not c.scheduled:
                c.schedule(s)
                change_log.add((c, "Scheduled"))
                
        elif not c.registered:
            if c.scheduled:
                c.unschedule(s)
                change_log.add((c, "Unscheduled"))
                
    return change_log

def restore_schedule(s, change_log, ignore): 
    for c in change_log:
        if c[0] in ignore:
            print("Successfully ignored course")
            continue
        if c[1] == "Scheduled":
            c[0].unschedule(s)
        elif c[1] == "Unscheduled":
            c[0].schedule(s)

def register(s):
    payload = {
        'Origin': 'https://webreg.usc.edu',
        'Upgrade-Insecure-Requests': '1',
    }
    s.get("https://webreg.usc.edu/Register")
    r = s.post("https://webreg.usc.edu/RegResp", data=payload)
    # print(r.text)
    if "Registration Failed" in r.text:
        print("Registration failed")
    else:
        print("Registration succeded")
        

In [121]:
# Login
import credentials

#session = usc_auth(credentials.get_username(), credentials.get_password())
session = webreg_login()
report = course_report(session)
#for c in report:
#    print(c.section)
print("Done.")


File exists! recovering session ...
Done.


In [122]:
change_log = simplify_schedule(session, report)

Unscheduled with: https://webreg.usc.edu/myCoursebin/SchdUnschRmv?act=UnSched&section=14025


In [123]:
for e in change_log:
    break
restore_schedule(session, change_log, [])

Scheduled with: https://webreg.usc.edu/myCoursebin/SchdUnschRmv?act=Sched&section=14025


In [139]:
class registrar:
    def __init__(self, desirable_combos):
        self.desirable_combos = desirable_combos
        print ("Registrar init")
    
    def auto_reg(self, s):
        report = course_report(s)
        print("Report len " + str(len(report)) )
        reg_courses = self.find_courses(report, self.target_sections)
        self.attempt_reg(s, reg_courses, report)
        
    def attempt_reg(self, s, reg_courses, all_courses):
        
        # setup
        change_log = simplify_schedule(s, all_courses)
        reg_needed = False;
        print(len(reg_courses))
        # add courses
        for course in reg_courses:
            print(course.section)
            if course.availible and not course.registered:
                course.schedule(s)
                reg_needed = True
        
        # register
        if reg_needed:
            print("Attempting registration ...")
            register(s)
            
                
        # cleanup
        print("Reg Courses Size: " + str(len(reg_courses)))
        restore_schedule(s, change_log, reg_courses)
        
    def find_courses(self, all_courses, sections):
        courses = set()
        for c in all_courses:
            if int(c.section) in sections:
                courses.add(c)
        return courses

reg = registrar({{[39625], [39626, 39627]}})

#
{{[], []} {} {} } 
#desired->requirements, choose 1 of each, priority to smaller indexs
reg.auto_reg(session)

Registrar init
Report len 22
Unscheduled with: https://webreg.usc.edu/myCoursebin/SchdUnschRmv?act=UnSched&section=11864
Unscheduled with: https://webreg.usc.edu/myCoursebin/SchdUnschRmv?act=UnSched&section=10202
Unscheduled with: https://webreg.usc.edu/myCoursebin/SchdUnschRmv?act=UnSched&section=14025
Unscheduled with: https://webreg.usc.edu/myCoursebin/SchdUnschRmv?act=UnSched&section=11863
2
11863
Scheduled with: https://webreg.usc.edu/myCoursebin/SchdUnschRmv?act=Sched&section=11863
11864
Scheduled with: https://webreg.usc.edu/myCoursebin/SchdUnschRmv?act=Sched&section=11864
Attempting registration ...
Registration succeded
Reg Courses Size: 2
Successfully ignored course
Successfully ignored course
Scheduled with: https://webreg.usc.edu/myCoursebin/SchdUnschRmv?act=Sched&section=10202
Scheduled with: https://webreg.usc.edu/myCoursebin/SchdUnschRmv?act=Sched&section=14025


In [20]:
for c in report:
    if int(c.section) == 49448:
        print(c.scheduled)
        

False
